In [1]:
gpx_filename = 'data-input/28may2021.gpx'

In [2]:
import os

from os.path import isfile, join, isdir, dirname, abspath
from os import listdir
import gpxpy
import gpxpy.gpx

import geopandas as gpd
from shapely.geometry import LineString


def parseActivityDataInGpxData(gpx =  None):
    if not gpx:
        return None

    activity = {}
    activity['time'] = gpx.time
    activity['link'] = gpx.link
    tracks = []
    for track_item in gpx.tracks:
        track = {}
        
        track['name'] = track_item.name
        track['type'] = track_item.type
        segments = []
        for segment_item in track_item.segments:
            segment = {}
            points = []
            for point_item in segment_item.points:
                point = []
                #print(point_item)
                #point.append(point.time)
                point.append(float(point_item.latitude))
                point.append(float(point_item.longitude))
                point.append(float(point_item.elevation))
                points.append(point)
            segment['points'] = points
            segments.append(segment)
        track['segments'] = segments
        tracks.append(track)
    activity['tracks'] = tracks
    
    return activity
    

def getActivityDataFromGpxFile(file_path):
    if not isfile(file_path):
        return None
    
    #print('file_path', file_path)
    file_handle = open(file_path, "r")
    gpx_data = gpxpy.parse(file_handle)
    file_handle.close()
    
    activity_data = parseActivityDataInGpxData(gpx_data)
    
    return activity_data


In [3]:
data_activity = getActivityDataFromGpxFile(gpx_filename)
points_full = data_activity['tracks'][0]['segments'][0]['points']
 
points = [point[:2] for point in points_full]
gdf = gpd.GeoDataFrame(geometry=[LineString(points)])

In [4]:
from ipyleaflet import Map, Polyline
from ipywidgets import Layout


import geopandas as gpd
from shapely.geometry import LineString, Point
from scipy.interpolate import splprep, splev
import numpy as np
import matplotlib.pyplot as plt


In [5]:
def simplify(gdf, tolerance=0.001):
    gdf_simplified = gdf.simplify(tolerance)
    print('Simplify Coords count: ', len(gdf.geometry.iloc[0].coords), ' -> ', len(gdf_simplified.geometry.iloc[0].coords))
    print('Tolerance: ', tolerance)
    print()
    
    return gdf_simplified


def gdf2line(_gdf, color='red'):
    return Polyline(locations=list(_gdf.geometry.iloc[0].coords), color=color, fill=False)


def gdf2centroid(_gdf):
    return list(_gdf.centroid[0].coords)[0]
    

def gdf2splines(_gdf):
    _points = list(_gdf.geometry.iloc[0].coords)
    
    # Fit spline through points
    tck, _ = splprep(np.array(_points).T, s=0.0)
    
    # Evaluate spline at a higher resolution
    u_new = np.linspace(0, 1, 1000)
    spline_points = splev(u_new, tck)
    
    # Convert spline points to Shapely Point objects
    point_geoms = [Point(x, y) for x, y in zip(*spline_points)]
    
    return gpd.GeoDataFrame(geometry=[LineString(point_geoms)])


def gdf_reverse_coords(_gdf):
    return gpd.GeoDataFrame({'geometry': [LineString([(c2,c1) for c1, c2 in list(_gdf.geometry.iloc[0].coords)])]})


def points2gdf(_points):
    return gpd.GeoDataFrame(geometry=[LineString(_points)])

def gdf2coords(_gdf):
    return list(_gdf.geometry.iloc[0].coords)
    

In [6]:
gdf1 = simplify(gdf,  tolerance=0.0000001)
gdf2 = simplify(gdf1, tolerance=0.000001)
gdf3 = simplify(gdf2, tolerance=0.00001)
gdf4 = simplify(gdf3, tolerance=0.0001)
gdf5 = simplify(gdf4, tolerance=0.0001)
gdf9 = simplify(gdf5, tolerance=0.01)

gdf_spline = gdf2splines(gdf9)


m = Map(center=gdf2centroid(gdf9), zoom=13, layout=Layout(width='100%', height='700px'))
m.add(gdf2line(gdf1, 'navy'))
m.add(gdf2line(gdf5, 'yellow'))

m.add(gdf2line(gdf9, 'red'))

m.add(gdf2line(gdf_spline, 'green'))
m


Simplify Coords count:  11589  ->  10969
Tolerance:  1e-07

Simplify Coords count:  10969  ->  7676
Tolerance:  1e-06

Simplify Coords count:  7676  ->  1977
Tolerance:  1e-05

Simplify Coords count:  1977  ->  386
Tolerance:  0.0001

Simplify Coords count:  386  ->  379
Tolerance:  0.0001

Simplify Coords count:  379  ->  13
Tolerance:  0.01



Map(center=[55.67781692843676, 36.71038865754297], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [8]:
from datetime import datetime

gdf_reverse_coords(gdf5).to_file(f'data-output/zvenigorod-route-{int(datetime.now().timestamp())}.geojson', driver='GeoJSON')


In [9]:
camera_points = [
    (55.563636, 36.592460),
    (55.575730, 36.530853),
    (55.608732, 36.548689),
    (55.634572, 36.640942),
    (55.707225, 36.652690),
    (55.705424, 36.714075),
    (55.705150, 36.807214),
    (55.692771, 36.824143),
    (55.701127, 36.932805),
]

gdf_camera_spliens = gdf2splines(points2gdf(camera_points))

m.add(gdf2line(gdf_camera_spliens, 'magenta'))
m

Map(bottom=656732.0, center=[55.67781692843676, 36.71038865754297], controls=(ZoomControl(options=['position',…

In [10]:
import string
import utils
from datetime import datetime

# Make json

js_file_template = string.Template('''
const routes = {
    target: [
    $target
],
    camera: [
    $camera
    ]
};
''')


text = js_file_template.substitute(
    target = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(gdf_spline))]),
    camera = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(gdf_camera_spliens))])
)

utils.write_file(f'zvenigorod-camera-{int(datetime.now().timestamp())}.js', text)


PosixPath('zvenigorod-camera-1712166271.js')

In [41]:
import math

st_cam_points = [
    (55.577952, 36.515800),
    (55.690757, 36.825998),
]
print(st_cam_points)
print()

dgf_st_cam = gpd.GeoDataFrame({'geometry': [LineString(st_cam_points)]})
print(dgf_st_cam)


delta = 0.02

st_target_points = [(round(p[0]+delta, 6), round(p[1]-delta, 6))for p in st_cam_points]
print(st_target_points)
print()

dgf_st_target = gpd.GeoDataFrame({'geometry': [LineString(st_target_points)]})
print(dgf_st_target)


m_st = Map(center=gdf2centroid(gdf9), zoom=11)
m_st.add(gdf2line(dgf_st_cam, 'cyan'))
m_st.add(gdf2line(dgf_st_target, 'lime'))
m_st



[(55.577952, 36.5158), (55.690757, 36.825998)]

                                            geometry
0  LINESTRING (55.57795 36.51580, 55.69076 36.82600)
[(55.597952, 36.4958), (55.710757, 36.805998)]

                                            geometry
0  LINESTRING (55.59795 36.49580, 55.71076 36.80600)


Map(center=[55.67781692843676, 36.71038865754297], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [42]:


text = js_file_template.substitute(
    target = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(dgf_st_target))]),
    camera = ',\n'.join([f'\t\t[{c[0]}, {c[1]}]' for c in gdf2coords(gdf_reverse_coords(dgf_st_cam))])
)

utils.write_file(f'data-output/st-line-{int(datetime.now().timestamp())}.js', text)

PosixPath('data-output/st-line-1712168373.js')